In [1]:
import requests
from pprint import pprint
import pandas as pd
from pandas import ExcelWriter
import io, time, json



headers = {'Content-Type': 'application/json', 'user-key': '592789033e4cf0e384b54c1ec424c706'}
offset = 0
restaurantNameList = []
ratingList = []
ratingTextList = []
votesList = []
totalData = []
urlList = []
addressList = []
while offset < 100:
    url = 'https://developers.zomato.com/api/v2.1/search?entity_id=291&entity_type=city&start='+str(offset)+'&count=20'
    Response = requests.get(url , headers=headers)
    offset += 20
    Response_Json = Response.json()
    Array_length = Response_Json['restaurants']
    for i in range(len(Array_length)):
        totalData.append(Response_Json['restaurants'][i])
        restaurantNameList.append(Response_Json['restaurants'][i]['restaurant']['name'])
        ratingList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['aggregate_rating'])
        ratingTextList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['rating_text'])
        votesList.append(Response_Json['restaurants'][i]['restaurant']['user_rating']['votes'])
        urlList.append(Response_Json['restaurants'][i]['restaurant']['url'])
        addressList.append(Response_Json['restaurants'][i]['restaurant']['location']['address'])	
df1 = pd.DataFrame({'AllData':totalData})
df2 = pd.DataFrame({'Restaurant Name':restaurantNameList,'Rating':ratingList, 'Rating Text':ratingTextList, 'Votes':votesList, 'Url':urlList, 'Address': addressList})
writer = pd.ExcelWriter('Final_Data.xlsx' , engine='xlsxwriter')
df1.to_excel(writer,'Sheet1',index=False)
df2.to_excel(writer,'Sheet2',index=False)
writer.save()


In [ ]:
def read_api_key(filepath):
    """
    Read the Yelp API Key from file.
    
    Args:
        filepath (string): File containing API Key
    Returns:
        api_key (string): The API Key
    """
    
    # feel free to modify this function if you are storing the API Key differently
    with open('api_key.txt', 'r') as f:
        return f.read().replace('\n','')

In [ ]:
def all_restaurants(api_key, query):
    """
    Retrieve ALL the restaurants on Yelp for a given query.

    Args:
        query (string): Search term

    Returns:
        results (list): list of dicts representing each business
    """
    
    # Write solution here
    offsetno=0
    total =0
    output=[]
    headers = {        
        "authorization": 'Bearer %s' % api_key, # for the yelp API     
    }
    while (offsetno<=total):
        params = { #parameters are case sensitive!
            "location": query,
            "limit": 20,
            "offset": offsetno,
            "categories": "restaurants"
            
        }
    
        response = requests.get('https://api.yelp.com/v3/businesses/search',
                            headers=headers, params=params)
        result = response.json()
        try:
            output.append( result['businesses'])
        except KeyError:
            print(result)
            return output
        offsetno =offsetno + 20
        total = result['total']
        print (offsetno)
        print (len(output))
    return(output)

In [ ]:
api_key = read_api_key('api_key.txt')
data = all_restaurants(api_key, 'Miami')
new_data = {}
new_data["name"]=[]
new_data["rating"]=[]
new_data["price"]=[]
new_data["review_count"]=[]
new_data["url"]=[]
new_data["category"]=[]
new_data["zip"]=[]
alldata=[]
for i in data:
    for each in i:
        new_data["name"].append(each["name"])
        new_data["rating"].append(each["rating"])
        new_data["url"].append(each["url"])
        try:
            new_data["price"].append(each["price"])
        except KeyError:
            new_data["price"].append(None)

        new_data["review_count"].append(each["review_count"])
        new_data["category"].append(each["categories"][0]["title"])
        new_data["zip"].append(each["location"]["zip_code"])
        alldata.append(each)
df = pd.DataFrame(new_data)
df.to_csv("Miami_Yelp.csv",index=False)
df_all = pd.DataFrame(alldata)
df_all.to_csv("Miami_raw.csv",index=False)
df = pd.read_excel("/Users/srossgupta/Desktop/Data Focussed Python/HungryGator/DataFiles/Project_prototype_datafiles.xlsx",sheet_name="MergedData")

In [ ]:
data = pd.read_csv('TripAdvisor_Data.csv')
cleaned_data = data[data["City"]=="Miami"]
cleaned_data = cleaned_data.loc[:,["Restaurant ID", "Restaurant URL","Name","Address","Ranking","Total Review"]]
path = r"C:/Users/devdi/Desktop/CMU/Final Mini/Python for Data Science/Project/Data.xlsx"
writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
data.to_excel(writer, sheet_name = 'TripAdvisor_raw')
cleaned_data.to_excel(writer, sheet_name = 'TripAdvisor_clean')
writer.save()
writer.close()

